In [2]:
import pandas as pd
import json
import random
import os

In [2]:
# imgs = {}
# for img in working['images']:
#     id = img['id']
#     file_list = img['file_name'][:-4].split('_')
#     file = '_'.join(file_list[:-1])
#     frame_num = int(file_list[-1])
# #     if id == 11 or id == 6:
# #         print(img)
#     if file in imgs:
#         imgs[file].append((id,frame_num))
#     else:
#         imgs[file] = [(id,frame_num)]
# imgs

In [3]:
def grouper(iterable, max_dist):
    prev = None
    group = []
    for item in iterable:
        if not prev or item[1] - prev <= max_dist:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item[1]
    if group:
        yield group

# numbers = [123, 124, 128, 160, 167, 213, 215, 230, 245, 255, 257, 400, 401, 402, 430]
# dict(enumerate(grouper(numbers,50), 1))

In [4]:
# img_ids_to_keep = []
# for vid in imgs:
#     clumps = dict(enumerate(grouper(imgs[vid],12),1))
#     for c in clumps:
#         rand_sample = clumps[c][random.randint(0,len(clumps[c])-1)]
#         print(vid, c, rand_sample, clumps[c])
#         img_ids_to_keep.append(rand_sample[0])
# sorted(img_ids_to_keep)

In [4]:
def unique_instances(input_json):
    imgs = {}
    for img in input_json['images']:
        id = img['id']
        file_list = img['file_name'][:-4].split('_')
        file = '_'.join(file_list[:-1])
        frame_num = int(file_list[-1])
        if file in imgs:
            imgs[file].append((id,frame_num))
        else:
            imgs[file] = [(id,frame_num)]
    
    img_ids_to_keep = []
    for vid in imgs:
        max_dist = 150 if 'reduced' in vid else 1200
        clumps = dict(enumerate(grouper(imgs[vid],max_dist),1))
        for c in clumps:
            rand_sample = clumps[c][random.randint(0,len(clumps[c])-1)]
            print(vid, c, rand_sample, clumps[c])
            img_ids_to_keep.append(rand_sample[0])
    return sorted(img_ids_to_keep)

In [6]:
# unique_instances(working)

In [11]:
def thin_json_unique(working_json, save_here=True, save_to=None):
    
    working = json.load(open(working_json,'r'))
    
    if 'categories' in working:
        new_json = {'info':working['info'],'licenses':working['licenses'],'categories':working['categories'],'captions':[],'annotations':[],'images':[]}
    else:
        new_json = {'info':working['info'],'licenses':working['licenses'],'annotations':[],'images':[]}

    conversion_dict = {}
    for uid in unique_instances(working):
#         print(id, len(new_json['images'])+1)
        img = working['images'][uid-1]
        if uid != img['id']:
            print("{} != {}".format(uid,img['id']))
        img['id'] = len(new_json['images'])+1
        new_json['images'].append(img)
        if uid in conversion_dict:
            print('{} already in conversion_dict with val {}, trying to add {}'.format(uid,conversion_dict[uid],img['id']))
        conversion_dict[uid] = img['id']

    if 'captions' in working:  
        for cap in working['captions']:
            if cap['image_id'] in conversion_dict:
                cap['image_id'] = conversion_dict[cap['image_id']]
                cap['id'] = len(new_json['captions'])+1
                new_json['captions'].append(cap)

    for ann in working['annotations']:
        if ann['image_id'] in conversion_dict:
            ann['image_id'] = conversion_dict[ann['image_id']]
            ann['id'] = len(new_json['annotations'])+1
            new_json['annotations'].append(ann)

    if save_here is True:
        split = os.path.splitext(working_json)
        save_path = split[0]+"-unique.json"
        with open(save_path,"w") as f:
            json.dump(new_json,f)
    elif save_to is not None:
        with open(save_to,"w") as f:
            json.dump(new_json,f)
            
    return new_json

In [8]:
# fp = '/mnt/ssd1/processed/industry-data/project-max/ml/cloud-factory-data/with-bg/binary-datasets/forwards/IP/earl-originals/ip-bg-forwards-train.json'
# working = json.load(open(fp,'r'))
# for i, img in enumerate(working['images']):
#     print(i,img['id']-1)

In [12]:
thin_json_unique('/mnt/ssd1/processed/industry-data/project-max/ml/cloud-factory-data/with-bg/binary-datasets/forwards/IP/earl-originals/ip-bg-forwards-train.json')

410375NSCC_280416_PR_U 1 (347, 2377) [(347, 2377)]
1504164_16112016_U_141430 1 (225, 3531) [(225, 3531)]
2000306694WCC_110817_PR_U 1 (22, 642) [(21, 176), (22, 642), (23, 363), (24, 330)]
193063MCC_071116_PR_U 1 (238, 4087) [(238, 4087)]
193063MCC_071116_PR_U 2 (239, 9497) [(239, 9497), (240, 8334)]
5184147WCC_010516_PR_D 1 (137, 2441) [(136, 2183), (137, 2441), (138, 3248)]
1600774NSCC_260717_PR_D 1 (326, 785) [(324, 562), (325, 439), (326, 785), (327, 461)]
162410MCC_210217_PR_D 1 (2972, 840) [(2963, 522), (2964, 528), (2965, 534), (2966, 540), (2967, 546), (2968, 552), (2969, 558), (2970, 564), (2971, 570), (2972, 840), (2973, 846), (2974, 852), (2975, 858), (2976, 864), (2977, 870), (2978, 876), (2979, 882), (2980, 888), (2981, 1140)]
162410MCC_210217_PR_D 2 (2992, 2532) [(2982, 2472), (2983, 2478), (2984, 2484), (2985, 2490), (2986, 2496), (2987, 2502), (2988, 2508), (2989, 2514), (2990, 2520), (2991, 2526), (2992, 2532), (2993, 2538), (2994, 2544), (2995, 2550), (2996, 2556), (29

{'annotations': [{'annotation_type': 'point',
   'area': 10.0,
   'bbox': [30, 30, 1.0, 1.0],
   'category_id': 1,
   'date': None,
   'deleted': False,
   'id': 1,
   'image_id': 1,
   'iscrowd': 0,
   'occluded': False,
   'segmentation': [30, 30, 30, 31, 31, 31, 30],
   'verified': False},
  {'annotation_type': 'point',
   'area': 10.0,
   'bbox': [30, 30, 1.0, 1.0],
   'category_id': 1,
   'date': None,
   'deleted': False,
   'id': 2,
   'image_id': 2,
   'iscrowd': 0,
   'occluded': False,
   'segmentation': [30, 30, 30, 31, 31, 31, 30],
   'verified': False},
  {'annotation_type': 'point',
   'area': 10.0,
   'bbox': [30, 30, 1.0, 1.0],
   'category_id': 1,
   'date': None,
   'deleted': False,
   'id': 3,
   'image_id': 3,
   'iscrowd': 0,
   'occluded': False,
   'segmentation': [30, 30, 30, 31, 31, 31, 30],
   'verified': False},
  {'annotation_type': 'point',
   'area': 10.0,
   'bbox': [30, 30, 1.0, 1.0],
   'category_id': 1,
   'date': None,
   'deleted': False,
   'id': 4